# Combining multiple `Datasets` 

## Objectives:

Show how to combine multiple ECCO v4 state estimate `Datasets` after loading.

## Loading multiple `Datasets`  

In previous tutorials we've loaded single lat-lon-cap NetCDF tile files for ECCO state estimate variables and model grid parameters.  Here we will demonstrate merging the resulting `Datasets` together.  Some benefits of merging `Datasets` include having a tidier workspace and simplying subsetting operations, the subject of a future tutorial.  

First, we'll load three ECCOv4 NetCDF state estimate variables and the model grid.  For this exercise use the ECCOv4 NetCDF files in Format 2 (one NetCDF file corresponds to one variable and one year).  

First let's define our environment

In [17]:
import numpy as np
import xarray as xr
import sys
import matplotlib.pyplot as plt
import json

In [2]:
## Import the ecco_v4_py library into Python
## =========================================

## -- For the ECCO Summer School, use 
import ecco_v4_py as ecco

## -- If ecco_v4_py is not installed in your local Python library, 
##    tell Python where to find it.  For example, if your ecco_v4_py
##    files are in /Users/ifenty/ECCOv4-py/ecco_v4_py, then use:

# sys.path.append('/Users/ifenty/ECCOv4-py')
# import ecco_v4_py as ecco

In [3]:
## Tell Python where to find some ECCO NetCDF 
## yearly-aggregated fields (Format 1)
## ===================================

## -- For the ECCO Summer School, use
base_dir = '/work/projects/aci/ECCO/community/ECCO/ECCOv4/Release3p1'

## -- If files are on a local machine, use something like 
#base_dir = '/Users/ifenty/'

ECCO_dir = base_dir + '/eccov4r3_native_grid_netcdf/'

### *c* point: ``SSH``

In [4]:
help(ecco.load_ecco_var_from_years_nc)

Help on function load_ecco_var_from_years_nc in module ecco_v4_py.tile_io:

load_ecco_var_from_years_nc(data_dir, var_to_load, years_to_load='all', tiles_to_load='all', k_subset=[], dask_chunk=True, less_output=True)
    Loads one or more ECCOv4 NetCDF state estimate variable in the
    format of one file per variable per year.  All 13 tiles of the 
    lat-lon-cap (llc) grid are present in this type of file.
    
    Files in this format have names like 
        /eccov4r3_native_grid_netcdf/mon_mean/THETA_2010.nc
        /eccov4r3_native_grid_netcdf/day_mean/THETA_2010.nc
        /eccov4r3_native_grid_netcdf/mon_snapshot/THETA_2010.nc
    
    Simply point this routine at a directory with one or more 
    of these files and one or more years of var_to_load variables will
    be loaded. 
    * Used repeatedly by recursive_load_ecco_var_from_years_nc *
    
    
    Parameters
    ----------
    data_dir : str
        path to a directory within which we will look for NetCDF tile files
 

In [5]:
SSH_dir= ECCO_dir + '/mon_mean/SSH/'  
var = 'SSH'

# make use of the ``load_ecco_var_from_years_nc`` routine to 
# load a single variable 
ecco_dataset_A = ecco.load_ecco_var_from_years_nc(SSH_dir, \
                                                  var, \
                                                  tiles_to_load=[0,1,2,3],\
                                                  years_to_load=[2010])

to see the data variables in a dataset, use ``.data_vars``:

In [6]:
ecco_dataset_A.data_vars

Data variables:
    SSH      (time, tile, j, i) float32 dask.array<shape=(12, 4, 90, 90), chunksize=(12, 4, 90, 90)>

As expected, *ecco_dataset_A* has one ``data variable``, *SSH*, which has dimensions **i**, **j**, **tile**, and **time**.
Its coordinates:

In [7]:
ecco_dataset_A.SSH.coords

Coordinates:
  * j        (j) int32 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * i        (i) int32 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
    XC       (tile, j, i) float32 dask.array<shape=(4, 90, 90), chunksize=(4, 90, 90)>
    YC       (tile, j, i) float32 dask.array<shape=(4, 90, 90), chunksize=(4, 90, 90)>
    rA       (tile, j, i) float32 dask.array<shape=(4, 90, 90), chunksize=(4, 90, 90)>
  * tile     (tile) int32 0 1 2 3
    iter     (time) int32 dask.array<shape=(12,), chunksize=(12,)>
  * time     (time) datetime64[ns] 2010-01-16T12:00:00 ... 2010-12-16T12:00:00

### *u* point: ``ADVx_TH``

``ADVx_TH`` is the horizontal advective flux of potential temperature in each tile's $x$ direction.

In [8]:
ADVx_dir= ECCO_dir + '/mon_mean/ADVx_TH/'  
var = 'ADVx_TH'

# make use of the ``load_ecco_var_from_years_nc`` routine to 
# load a single variable 
ecco_dataset_B= ecco.load_ecco_var_from_years_nc(ADVx_dir, \
                                                 var, \
                                                 years_to_load=2010).load();
ecco_dataset_B.attrs=[]

ecco_dataset_B.data_vars

Data variables:
    ADVx_TH  (time, tile, k, j, i_g) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

*ecco_dataset_A* has one ``data variable``, *ADVx_TH*, which has dimensions **i_g**, **j**, **k** **tile**, and **time**.  *ADVx_TH* is a three-dimensional field.

Notice that the coordinates of *ecco_dataset_B* is distinct from *ecco_dataset_A*.  Specifically, *ecco_dataset_B* has a **k** dimension, and several new non-dimension coords such as **drF** and **dyG**

In [9]:
ecco_dataset_B.coords

Coordinates:
  * i_g        (i_g) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * j          (j) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k          (k) int32 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
    Z          (k) float32 -5.0 -15.0 -25.0 -35.0 ... -5039.25 -5461.25 -5906.25
    PHrefC     (k) float32 49.05 147.15 245.25 ... 49435.043 53574.863 57940.312
    drF        (k) float32 10.0 10.0 10.0 10.0 10.0 ... 387.5 410.5 433.5 456.5
    dxC        (tile, j, i_g) float32 15583.418 15588.104 ... 23406.256
    rAw        (tile, j, i_g) float32 361699460.0 362790240.0 ... 364760350.0
    dyG        (tile, j, i_g) float32 23210.262 23273.26 ... 15595.26 15583.685
    hFacW      (tile, k, j, i_g) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * tile       (tile) int32 0 1 2 3 4 5 6 7 8 9 10 11 12
    time_bnds  (time, nv) datetime64[ns] 2010-01-01 2010-02-01 ... 2011-01-01
    iter       (time) int32 158532 159204 159948 160668 ... 

### *v* point: ``ADVy_TH``

``ADVy_TH`` is the horizontal advective flux of potential temperature in each tile's $y$ direction.

In [10]:
ADVy_dir= ECCO_dir + '/mon_mean/ADVy_TH/'  
var = 'ADVy_TH'

ecco_dataset_C= ecco.load_ecco_var_from_years_nc(ADVy_dir, \
                                                 var, \
                                                 years_to_load=2010).load();
ecco_dataset_C.attrs=[]
ecco_dataset_C.data_vars

Data variables:
    ADVy_TH  (time, tile, k, j_g, i) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

*ecco_dataset_C* has one ``data variable``, *ADVy_TH*, which has dimensions **i**, **j_g**, **k** **tile**, and **time**.  *ADVy_TH* is a three-dimensional field.

### Examining the dimensions and coordinates of these `Datasets`

Each of our three `Datasets` contain a single  `DataArray`.  Each of these `DataArray` objects has different horizontal dimension labels.  

* **i** and **j** for *SSH*
* **i_g** and **j** for *ADVx_TH*
* **i** and **j_g** for *ADVy_TH*


In [11]:
print(ecco_dataset_A.data_vars)
print(ecco_dataset_B.data_vars)
print(ecco_dataset_C.data_vars)

Data variables:
    SSH      (time, tile, j, i) float32 dask.array<shape=(12, 4, 90, 90), chunksize=(12, 4, 90, 90)>
Data variables:
    ADVx_TH  (time, tile, k, j, i_g) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Data variables:
    ADVy_TH  (time, tile, k, j_g, i) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0


## Merging multiple `Datasets`  from state estimate variables

Using the `xarray` method ``merge`` we can create a single `Dataset` with multiple `DataArrays`.  

In [12]:
# merge together
ecco_dataset_ABC = xr.merge([ecco_dataset_A, ecco_dataset_B, ecco_dataset_C]).load()

### Examining the merged `Dataset`

As before, let's look at the contents of the new merged `Dataset`

In [13]:
ecco_dataset_ABC

<xarray.Dataset>
Dimensions:    (i: 90, i_g: 90, j: 90, j_g: 90, k: 50, nv: 2, tile: 13, time: 12)
Coordinates:
  * tile       (tile) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j          (j) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * i          (i) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
    XC         (tile, j, i) float32 -111.60647 -111.303 -110.94285 ... nan nan
    YC         (tile, j, i) float32 -88.24259 -88.382515 -88.52242 ... nan nan
    rA         (tile, j, i) float32 362256450.0 363300960.0 ... nan nan
    time_bnds  (time, nv) datetime64[ns] 2010-01-01 2010-02-01 ... 2011-01-01
    iter       (time) int32 158532 159204 159948 160668 ... 165084 165804 166548
  * time       (time) datetime64[ns] 2010-01-16T12:00:00 ... 2010-12-16T12:00:00
  * i_g        (i_g) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k          (k) int32 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
    Z          (k) float32 -5.0 -15.0 -25.0


#### 1. Dimensions
`Dimensions:   (i: 90, i_g: 90, j: 90, j_g: 90, k: 50, nv: 2, tile: 13, time: 12)`

*ecco_dataset_merged* is a container of ``Data Arrays`` and as such it lists all of the unique dimensions its ``Data Arrays``. In other words, *Dimensions* shows all of the dimensions used by its variables. 

#### 2. Dimension Coordinates
```
Coordinates:
Coordinates:
  * j          (j) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * i          (i) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * tile       (tile) int32 0 1 2 3 4 5 6 7 8 9 10 11 12
  * time       (time) datetime64[ns] 2010-01-16T12:00:00 ... 2010-12-16T12:00:00
  * i_g        (i_g) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k          (k) int32 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * j_g        (j_g) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
``` 

Notice that the **tile** and **time** coordinates are unchanged.  ``merge`` recognizes identical coordiantes and keeps them.

#### 3. Non-Dimension Coordinates
```
Coordinates:
    XC         (tile, j, i) float32 -111.60647 -111.303 ... -111.86579
    YC         (tile, j, i) float32 -88.24259 -88.382515 ... -88.07871 -88.10267
    rA         (tile, j, i) float32 362256450.0 363300960.0 ... 361119100.0
    time_bnds  (time, nv) datetime64[ns] 2010-01-01 2010-02-01 ... 2011-01-01
    iter       (time) int32 158532 159204 159948 160668 ... 165084 165804 166548
    Z          (k) float32 -5.0 -15.0 -25.0 -35.0 ... -5039.25 -5461.25 -5906.25
    PHrefC     (k) float32 49.05 147.15 245.25 ... 49435.043 53574.863 57940.312
    drF        (k) float32 10.0 10.0 10.0 10.0 10.0 ... 387.5 410.5 433.5 456.5
    dxC        (tile, j, i_g) float32 15583.418 15588.104 ... 23406.256
    rAw        (tile, j, i_g) float32 361699460.0 362790240.0 ... 364760350.0
    dyG        (tile, j, i_g) float32 23210.262 23273.26 ... 15595.26 15583.685
    hFacW      (tile, k, j, i_g) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    rAs        (tile, j_g, i) float32 179944260.0 180486990.0 ... 364150620.0
    dxG        (tile, j_g, i) float32 15584.907 15589.316 ... 23142.107
    dyC        (tile, j_g, i) float32 11563.718 11593.785 ... 15578.138
    hFacS      (tile, k, j_g, i) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
```

The list of non-dimension coordinates is now much longer.  Like *Dimensions*, the non-dimension coordinates of the merged ``Dataset`` contain all of the non-dimension coordinates of the ``Data Arrays``.


#### 4. Attributes

Notice that all of the high level `Dataset` attributes are gone!  Each of the three `Datasets` had different attributes and the `merge` routine simply drops them.  The attributes of the *Data variables* remain intact:

In [14]:
# (this json command makes Python dictionaries easier to read)
print(json.dumps(ecco_dataset_ABC.SSH.attrs, indent=2,sort_keys=True))

{
  "long_name": "Surface Height Anomaly adjusted with global steric height change and sea-ice load",
  "standard_name": "sea_surface_height",
  "units": "m"
}


## Adding the model grid  `Dataset`

Let's use the ``merge`` routine to combine a `Dataset` of the model grid parameters with `output_merged`.

### Load the model grid parameters

In [15]:
# Load the llc90 grid parameters
grid_dir= ECCO_dir + '/grid'
grid_dataset = xr.open_dataset(grid_dir + '/ECCOv4r3_grid.nc')
grid_dataset.coords

Coordinates:
  * i          (i) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * i_g        (i_g) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * j          (j) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * j_g        (j_g) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k          (k) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * k_u        (k_u) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * k_l        (k_l) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * k_p1       (k_p1) int64 0 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
  * tile       (tile) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC         (tile, j, i) float32 ...
    YC         (tile, j, i) float32 ...
    XG         (tile, j_g, i_g) float32 ...
    YG         (tile, j_g, i_g) float32 ...
    CS         (tile, j, i) float32 ...
    SN         (tile, j, i) float32 ...
    Z          (k) float32 ...
    Zp1    

### Merge ``grid_all_tiles`` with ``output_merged``

In [16]:
ecco_dataset_ABCG= xr.merge([ecco_dataset_ABC, grid_dataset])
ecco_dataset_ABCG

<xarray.Dataset>
Dimensions:    (i: 90, i_g: 90, j: 90, j_g: 90, k: 50, k_l: 50, k_p1: 51, k_u: 50, nv: 2, tile: 13, time: 12)
Coordinates:
  * tile       (tile) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j          (j) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * i          (i) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
    XC         (tile, j, i) float32 -111.60647 -111.303 ... -111.86579
    YC         (tile, j, i) float32 -88.24259 -88.382515 ... -88.07871 -88.10267
    rA         (tile, j, i) float32 362256450.0 363300960.0 ... 361119100.0
    time_bnds  (time, nv) datetime64[ns] 2010-01-01 2010-02-01 ... 2011-01-01
    iter       (time) int32 158532 159204 159948 160668 ... 165084 165804 166548
  * time       (time) datetime64[ns] 2010-01-16T12:00:00 ... 2010-12-16T12:00:00
  * i_g        (i_g) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k          (k) int32 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
    Z          (k

### Examining the merged `Dataset`

The result of this last merge is a single `Dataset` with 3 *Data variables*, and a complete set of model grid parameters (distances and areas).

## Merging and memory

Merging `Datasets` together does not make copies of the data in memory.  Instead, merged `Datasets` are in fact just a reorganized collection of pointers.  You may want to delete the original variables to clear your namespace, but it is not necessary.

## Summary

Now you know how to merge multiple `Datasets` using the `merge` command.  We demonstrated merging of `Datasets` constructed from three different variables types and the model grid parameters.